In [1]:
!pip install gtts

In [ ]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch_directml  # Import torch-directml for DirectML device
import torch  # Import PyTorch
from gtts import gTTS  # Import Google Text-to-Speech
import os  # For playing the sound file



In [ ]:
dml_device = torch_directml.device()


In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(dml_device)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")


In [ ]:
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
    # Add more languages as needed
}

def translate(text, source_lang, target_lang):
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(dml_device)  # Send input tensors to DirectML device
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    # Create the TTS audio using gTTS
    tts = gTTS(text=translated_text, lang=languages[target_lang])
    audio_file = "translated_speech.mp3"
    tts.save(audio_file)
    
    return translated_text, audio_file

with gr.Blocks() as app:
    gr.Markdown("# Multilingual Translation Service")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")
    audio_output = gr.Audio(label="Pronunciation")
    translate_btn = gr.Button("Translate")

    translate_btn.click(fn=translate, inputs=[input_text, source_lang, target_lang], outputs=[output_text, audio_output])

app.launch(inline=False, share=True)
